# Data Understanding
In deze notebook gaan we een blik werpen op de waardes van de dataset, hoeveel afbeeldingen per klasse zijn er en welke resolutie / ratio?

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision import datasets, models, transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import DataLoader, random_split

from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from IPython.display import FileLink, FileLinks

from collections import defaultdict
from fractions import Fraction

import random
import time

### Data Understanding

In [4]:
def count_images_in_folders(dataset_path):
    # Dictionary om het aantal afbeeldingen per klasse op te slaan
    class_counts = {}

    # Overloop alle mappen in de dataset
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        
        # Controleer of het een map is
        if os.path.isdir(class_path):
            # Tel het aantal afbeeldingsbestanden in de klassemap
            num_images = len([name for name in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, name))])

            # Voeg het aantal toe aan de dictionary
            class_counts[class_name] = num_images

    return class_counts

In [5]:
# Instellingen
data_dir = '..\\images\\selfmade_data'
class_counts = count_images_in_folders(data_dir)
num_classes = len(class_counts)

In [6]:
# Print het aantal afbeeldingen per klasse
for class_name, num_images in class_counts.items():
    print(f"Klasse '{class_name}' heeft {num_images} afbeeldingen.")

Klasse 'Kiwi' heeft 1125 afbeeldingen.
Klasse 'Banaan' heeft 922 afbeeldingen.
Klasse 'sinaasappel_mandarijn' heeft 1344 afbeeldingen.
Klasse 'Tomaat' heeft 980 afbeeldingen.
Klasse 'Appel' heeft 1107 afbeeldingen.
Klasse 'Peer' heeft 1032 afbeeldingen.


In het geval dat de klasse verdeling te groot is probeer dit gelijk te stellen door over- undersampling (met andere data augementatie op de dubbele afbeeldingen). 

In [8]:
def categorize_resolutions(dataset_path):
    # Dictionary voor het opslaan van de aantallen in specifieke categorieën
    resolution_categories = {
        "smaller_than_224": 0,
        "exact_224": 0,
        "exact_416": 0,
        "larger_than_416": 0,
        "between_224_and_415": 0
    }

    # Loop door alle mappen in de dataset
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)

        if os.path.isdir(class_path):
            for filename in os.listdir(class_path):
                file_path = os.path.join(class_path, filename)

                if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
                    try:
                        with Image.open(file_path) as img:
                            width, height = img.size

                            if width < 224 or height < 224:
                                resolution_categories["smaller_than_224"] += 1
                            elif width == 224 and height == 224:
                                resolution_categories["exact_224"] += 1
                            elif width == 416 and height == 416:
                                resolution_categories["exact_416"] += 1
                            elif width > 416 or height > 416:
                                resolution_categories["larger_than_416"] += 1
                            else:
                                resolution_categories["between_224_and_415"] += 1
                    except Exception as e:
                        print(f"Fout bij het openen van {file_path}: {e}")
    
    return resolution_categories

In [9]:
resolution_categories = categorize_resolutions(data_dir)

In [10]:
# Print de aantallen afbeeldingen per resolutiecategorie
for category, count in resolution_categories.items():
    print(f"Resolutie categorie '{category}': {count} afbeeldingen.")

Resolutie categorie 'smaller_than_224': 0 afbeeldingen.
Resolutie categorie 'exact_224': 0 afbeeldingen.
Resolutie categorie 'exact_416': 0 afbeeldingen.
Resolutie categorie 'larger_than_416': 6510 afbeeldingen.
Resolutie categorie 'between_224_and_415': 0 afbeeldingen.


    We kunnen hier twee keuzes maken, de standaard ResNet50 resolutie gebruiken (224x224) of een eigen resolutie (416x416) om meer details te behouden, dit is de resolutie van de webcam.

In [12]:
def categorize_aspect_ratios(dataset_path):
    # Definieer de aspect ratio categorieën
    categories = {
        "almost_square": 0,      # verhoudingen dichtbij 1:1
        "little_wide": 0,        # verhouding tussen 1:1 en 16:9
        "wide": 0,               # verhoudingen dichtbij 16:9
        "tall": 0,               # verhoudingen meer richting portret (b.v. 9:16)
        "extremely_wide": 0,     # verhoudingen veel breder dan 16:9
        "extremely_tall": 0      # verhoudingen veel hoger dan 9:16
    }

    # Loop door alle mappen in de dataset
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)

        if os.path.isdir(class_path):
            for filename in os.listdir(class_path):
                file_path = os.path.join(class_path, filename)

                if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')):
                    try:
                        with Image.open(file_path) as img:
                            width, height = img.size
                            ratio = float(width) / float(height)
                            
                            # Categoriseer de aspect ratio
                            if 0.9 <= ratio <= 1.1:
                                categories["almost_square"] += 1
                            elif 1.1 <= ratio < 1.4:
                                categories["little_wide"] += 1
                            elif 1.4 <= ratio <= 2:
                                categories["wide"] += 1
                            elif 0.5 <= ratio < 0.9:
                                categories["tall"] += 1
                            elif ratio > 2:
                                categories["extremely_wide"] += 1
                            elif ratio < 0.5:
                                categories["extremely_tall"] += 1
                    except Exception as e:
                        print(f"Fout bij het openen van {file_path}: {e}")
    return categories

In [13]:
aspect_ratio_categories = categorize_aspect_ratios(data_dir)

In [14]:
# Print het aantal afbeeldingen per aspect ratio categorie
for category, count in aspect_ratio_categories.items():
    print(f"Categorie '{category}': {count} afbeeldingen.")

Categorie 'almost_square': 0 afbeeldingen.
Categorie 'little_wide': 0 afbeeldingen.
Categorie 'wide': 6510 afbeeldingen.
Categorie 'tall': 0 afbeeldingen.
Categorie 'extremely_wide': 0 afbeeldingen.
Categorie 'extremely_tall': 0 afbeeldingen.


    Hieruit is te baseren dat we de extreme gevallen weg kunnen halen en extra borders toevoegen om de resterende afbeeldingen voor we deze naar de juiste resolutie brengen. (In het huidige project werken we alleen met een 1:1 resolutie)

#### Image Processing
Vanuit hier kan je te werk gaan met het aanpassen van de dataset.
Bekijk `Image Processing` voor de aanpassingen op de data zelf en `Under-Oversampling` voor het aanpassen van de klasse verdeling.

In [17]:
FileLink('Image Processing.ipynb')

E:\Studie\Stage\Computer-Vision-Showcase-SUPERP\code\Image Processing.ipynb

In [18]:
FileLink('over-under sampling.ipynb')

E:\Studie\Stage\Computer-Vision-Showcase-SUPERP\code\over-under sampling.ipynb

Normaal gesproken hoef je het model niet opnieuw in te laden als je dit al hebt gedaan door het trainen van het model maar op het moment dat je al het .pth bestand hebt en dit niet nog een keer wilt doen kan je op deze wijze het model inladen en gebruiken.